In [1]:
import numpy as np
import os, sys
sys.path.append('../')
from src.datasets.soccernet_generic_full_games_v2 import soccernet_dataset_generic
from src.utils.helper import samples_by_language
import src.utils.training_helper as training_helper
import torch
from torch.utils.data import Dataset, DataLoader

/home/oarongve/.local/share/virtualenvs/project-daredevil-8eBKzQn6/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
root_dir = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/"
train_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Train_300.npy"
valid_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Valid_100.npy"
test_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Test_100.npy"


In [3]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
%%bash
nvidia-smi

Mon Dec 14 22:06:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.165.02   Driver Version: 418.165.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:E7:00.0 Off |                    0 |
| N/A   38C    P0    70W / 350W |   2743MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
langpath_train = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/train_lang_dict.json'
langpath_valid = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/valid_lang_dict.json'
langpath_test = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/test_lang_dict.json'

samples_train_all = samples_by_language(langpath_train,train_list,'all')
samples_valid_all = samples_by_language(langpath_valid,valid_list,'all')
samples_test_all = samples_by_language(langpath_test,test_list,'all')


In [6]:
train_set_all = soccernet_dataset_generic(npy_file=train_list,root_dir=root_dir,lang='all',lang_dict=langpath_train)
valid_set_all = soccernet_dataset_generic(npy_file=valid_list,root_dir=root_dir,lang='all',lang_dict=langpath_valid)


In [7]:
train_set_all.load_waves()
valid_set_all.load_waves()

train_set_all.generate_mel_spectrograms(load_features=True)
valid_set_all.generate_mel_spectrograms(load_features=True)

train_set_all.load_resnet_features()
valid_set_all.load_resnet_features()

train_set_all.load_audio_resnet()
valid_set_all.load_audio_resnet()

100%|██████████| 100/100 [00:01<00:00, 77.80it/s]


In [10]:
resnet_visual = train_set_all[0]['resnet_features']
resnet_audio = train_set_all[0]['resnet_audio_feature']

In [62]:
for i in range(len(train_set_all)):
    resnet_visual = train_set_all[i]['resnet_features']
    resnet_audio = train_set_all[i]['resnet_audio_feature']
    audio_shape, resnet_shape = resnet_audio.shape[0],resnet_visual.shape[0]

    if audio_shape != resnet_shape:
        cutoff = min(audio_shape,resnet_shape)
        print(cutoff)
        resnet_visual = resnet_visual[:cutoff,:]
        resnet_audio = resnet_audio[:cutoff,:]
    
    ## normalize
    resnet_visual = (resnet_visual - np.min(resnet_visual)) - (np.max(resnet_visual) - np.min(resnet_visual))
    resnet_audio = (resnet_audio - np.min(resnet_audio)) - (np.max(resnet_audio) - np.min(resnet_audio)) 
    
    half = train_set_all[i]['half']
    path = root_dir+train_set_all[i]['vidpath']
    savename = path+"/audio_resnet_combined_norm_h"+str(half)+".npy"
    combined = np.concatenate((resnet_visual,resnet_audio),axis=1)
    np.save(savename,combined)

5602
5397
5397
5586
5399
5375
5817
5680
4818
5672
5399
5530
5398
5604
5495
5399
5400
5399
5604
5888
5683
5593
5741
5721
5390
5695
5584
5398
5399
5397
5399
5398
5425
5912
5399
5381
5660
5399
5399
5399
5399
5399
5857
5482
5399
5957
5400
5825
5395
5649


# Validset

In [10]:
for i in range(len(valid_set_all)):
    resnet_visual = valid_set_all[i]['resnet_features']
    resnet_audio = valid_set_all[i]['resnet_audio_feature']
    audio_shape, resnet_shape = resnet_audio.shape[0],resnet_visual.shape[0]
    if audio_shape != resnet_shape:
        cutoff = min(audio_shape,resnet_shape)
        print(cutoff)
        resnet_visual = resnet_visual[:cutoff,:]
        resnet_audio = resnet_audio[:cutoff,:]
    
    ## normalize
    resnet_visual = (resnet_visual - np.min(resnet_visual)) - (np.max(resnet_visual) - np.min(resnet_visual))
    resnet_audio = (resnet_audio - np.min(resnet_audio)) - (np.max(resnet_audio) - np.min(resnet_audio)) 
    
    half = valid_set_all[i]['half']
    path = root_dir+valid_set_all[i]['vidpath']
    savename = path+"/audio_resnet_combined_norm_h"+str(half)+".npy"
    combined = np.concatenate((resnet_visual,resnet_audio),axis=1)
    np.save(savename,combined)

5397
5404
5508
5418
5396
5653
5984
5785
5399
5399
5397
5507
5319
5359


In [11]:
for i in range(len(valid_set_all)):
    resnet_visual = valid_set_all[i]['resnet_features']
    resnet_audio = valid_set_all[i]['resnet_audio_feature']
    audio_shape, resnet_shape = resnet_audio.shape[0],resnet_visual.shape[0]
    if audio_shape != resnet_shape:
        cutoff = min(audio_shape,resnet_shape)
        print(cutoff)
        resnet_visual = resnet_visual[:cutoff,:]
        resnet_audio = resnet_audio[:cutoff,:]
    
    ## normalize
    #resnet_visual = (resnet_visual - np.min(resnet_visual)) - (np.max(resnet_visual) - np.min(resnet_visual))
    #resnet_audio = (resnet_audio - np.min(resnet_audio)) - (np.max(resnet_audio) - np.min(resnet_audio)) 
    
    half = valid_set_all[i]['half']
    path = root_dir+valid_set_all[i]['vidpath']
    savename = path+"/audio_resnet_combined_h"+str(half)+".npy"
    combined = np.concatenate((resnet_visual,resnet_audio),axis=1)
    np.save(savename,combined)

5397
5404
5508
5418
5396
5653
5984
5785
5399
5399
5397
5507
5319
5359


# HERE: Test set

In [12]:

test_set_all = soccernet_dataset_generic(npy_file=test_list,root_dir=root_dir,lang='all',lang_dict=langpath_test)
test_set_all.load_waves()
test_set_all.generate_mel_spectrograms(load_features=True)
test_set_all.load_resnet_features()
test_set_all.load_audio_resnet()


100%|██████████| 100/100 [00:01<00:00, 74.06it/s]


In [13]:
for i in range(len(test_set_all)):
    resnet_visual = test_set_all[i]['resnet_features']
    resnet_audio = test_set_all[i]['resnet_audio_feature']
    audio_shape, resnet_shape = resnet_audio.shape[0],resnet_visual.shape[0]
    if audio_shape != resnet_shape:
        cutoff = min(audio_shape,resnet_shape)
        print(cutoff)
        resnet_visual = resnet_visual[:cutoff,:]
        resnet_audio = resnet_audio[:cutoff,:]
    
    ## normalize
    resnet_visual = (resnet_visual - np.min(resnet_visual)) - (np.max(resnet_visual) - np.min(resnet_visual))
    resnet_audio = (resnet_audio - np.min(resnet_audio)) - (np.max(resnet_audio) - np.min(resnet_audio)) 
    
    half = test_set_all[i]['half']
    path = root_dir+test_set_all[i]['vidpath']
    savename = path+"/audio_resnet_combined_norm_h"+str(half)+".npy"
    combined = np.concatenate((resnet_visual,resnet_audio),axis=1)
    np.save(savename,combined)

6086
5679
5613
5399
5611
5511
5399
5778
5399
5398
5084
5854
5393
5398
5398
5396
5398
5398
5398
5621
5394
5393
5848
5399
5399


In [14]:
for i in range(len(test_set_all)):
    resnet_visual = test_set_all[i]['resnet_features']
    resnet_audio = test_set_all[i]['resnet_audio_feature']
    audio_shape, resnet_shape = resnet_audio.shape[0],resnet_visual.shape[0]
    if audio_shape != resnet_shape:
        cutoff = min(audio_shape,resnet_shape)
        print(cutoff)
        resnet_visual = resnet_visual[:cutoff,:]
        resnet_audio = resnet_audio[:cutoff,:]
    
    ## normalize
    #resnet_visual = (resnet_visual - np.min(resnet_visual)) - (np.max(resnet_visual) - np.min(resnet_visual))
    #resnet_audio = (resnet_audio - np.min(resnet_audio)) - (np.max(resnet_audio) - np.min(resnet_audio)) 
    
    half = test_set_all[i]['half']
    path = root_dir+test_set_all[i]['vidpath']
    savename = path+"/audio_resnet_combined_h"+str(half)+".npy"
    combined = np.concatenate((resnet_visual,resnet_audio),axis=1)
    np.save(savename,combined)

6086
5679
5613
5399
5611
5511
5399
5778
5399
5398
5084
5854
5393
5398
5398
5396
5398
5398
5398
5621
5394
5393
5848
5399
5399


In [59]:
np.save()

TypeError: _save_dispatcher() missing 2 required positional arguments: 'file' and 'arr'

In [20]:
resnet_audio.shape

(5400, 512)

In [17]:
resnet_visual.var()

1.6719587

In [9]:
def pad_by_wsize(w,data):
    
    right_idx = data.size()[1] - (data.size()[1] % 20)
    
    pleft = torch.zeros((128,(w//2)*40))
    pright = torch.zeros((128,(w//2)*40))
    data = torch.cat((pleft,data[:,:right_idx],pright),1)
    
    return data
    

In [10]:
def normalize_data(data,idx):
    left, right = idx
    sample = data[:,left:right]
    sample = ( sample - torch.min(sample) ) / ( torch.max(sample) - torch.min(sample))
    return sample

In [11]:
import torchvision
import torch.optim as optim
import torch.nn as nn
def generate_model():
    model = torchvision.models.resnet18(pretrained=True)
    model.conv1 = nn.Conv2d(1,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)
    model.fc = nn.Linear(512,4,bias=True)
    #resnet.features.conv0 = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    #resnet.classifier = nn.Linear(in_features=densenet.classifier.in_features, out_features=3,bias=True)


    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    return model, criterion,optimizer

In [12]:
class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

In [13]:
w = 8

In [14]:
model,_,_ = generate_model()
model.load_state_dict(torch.load("./best_model_"+str(w)+".pth"))
model.fc = Identity()

In [15]:
%%bash
nvidia-smi

Mon Dec 14 22:02:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.165.02   Driver Version: 418.165.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:E7:00.0 Off |                    0 |
| N/A   40C    P0    70W / 350W |   4036MiB / 32480MiB |      5%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [76]:
data = train_set_all[0]['ms']
data = pad_by_wsize(w,d)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [168]:
train_set_all[2]['half']

1

# Training set

In [170]:
%%time
for i in range(len(train_set_all)):
    data = train_set_all[i]['ms']
    data = pad_by_wsize(w,data)
    l = data.size()[1]
    model = model.to(device)
    model = model.eval()
    res = list()
    save_path = root_dir+train_set_all[i]['vidpath']+"/resnet18_audio_feat_v1_h"+str(train_set_all[i]['half'])+".npy"
    with torch.no_grad():
        for left_idx in range(0,l-(w*40),20):
            inputs = normalize_data(data,(left_idx,left_idx+(w*40))).to(device).unsqueeze(0).unsqueeze(0)
            res.append(model(inputs).cpu())
    k = np.vstack(res)
    np.save(save_path,k)
    print(f"saved for {save_path}")

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2016-03-19 - 18-00 Chelsea 2 - 2 West Ham/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2016-04-09 - 17-00 Swansea 1 - 0 Chelsea/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2016-04-09 - 17-00 Swansea 1 - 0 Chelsea/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2016-04-09 - 19-30 Manchester City 2 - 1 West Brom/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2016-04-09 - 19-30 Manchester City 2 - 1 West Brom/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2016-05-07 - 17-00 Sunderland 3 - 2 Chelsea/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/soun

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2016-2017/2017-01-02 - 18-00 Sunderland 2 - 2 Liverpool/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2016-2017/2017-01-02 - 18-00 Sunderland 2 - 2 Liverpool/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2016-2017/2017-01-14 - 20-30 Leicester 0 - 3 Chelsea/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2016-2017/2017-01-14 - 20-30 Leicester 0 - 3 Chelsea/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2016-2017/2017-01-15 - 19-00 Manchester United 1 - 1 Liverpool/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2016-2017/2017-01-15 - 19-00 Manchester United 1 - 1 Liverpool/resnet18_audio_feat_v1_h2.npy
saved for /work/oaro

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-14 - 21-45 Juventus 1 - 0 Monaco/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-14 - 21-45 Juventus 1 - 0 Monaco/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-21 - 21-45 Bayern Munich 6 - 1 FC Porto/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-21 - 21-45 Bayern Munich 6 - 1 FC Porto/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-22 - 21-45 Monaco 0 - 0 Juventus/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-22 - 

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2015-2016/2015-11-04 - 22-45 Maccabi Tel Aviv 1 - 3 FC Porto/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2015-2016/2015-11-04 - 22-45 Maccabi Tel Aviv 1 - 3 FC Porto/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2015-2016/2015-11-24 - 20-00 Zenit Petersburg 2 - 0 Valencia/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2015-2016/2015-11-24 - 20-00 Zenit Petersburg 2 - 0 Valencia/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2015-2016/2015-11-24 - 22-45 Barcelona 6 - 1 AS Roma/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-le

...

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2016-10-15 - 16-00 Napoli 1 - 3 AS Roma/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2016-10-23 - 16-00 Crotone 1 - 2 Napoli/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2016-10-23 - 16-00 Crotone 1 - 2 Napoli/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2016-10-23 - 21-45 AS Roma 4 - 1 Palermo/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2016-10-23 - 21-45 AS Roma 4 - 1 Palermo/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2016-10-25 - 21-45 Genoa 3 - 0 AC Milan/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/Soccer

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2017-03-19 - 22-45 AS Roma 3 - 1 Sassuolo/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2017-04-09 - 16-00 Bologna 0 - 3 AS Roma/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2017-04-09 - 16-00 Bologna 0 - 3 AS Roma/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2017-04-09 - 21-45 Lazio 0 - 3 Napoli/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2017-04-09 - 21-45 Lazio 0 - 3 Napoli/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/italy_serie-a/2016-2017/2017-04-15 - 16-00 AS Roma 1 - 1 Atalanta/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/Soccer

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2015-2016/2016-01-31 - 22-30 Real Madrid 6 - 0 Espanyol/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2015-2016/2016-01-31 - 22-30 Real Madrid 6 - 0 Espanyol/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2015-2016/2016-02-07 - 22-30 Granada CF 1 - 2 Real Madrid/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2015-2016/2016-02-07 - 22-30 Granada CF 1 - 2 Real Madrid/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2015-2016/2016-02-21 - 18-00 Malaga 1 - 1 Real Madrid/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2015-2016/2016-02-21 - 18-00 Malaga 1 - 1 Real Madrid/resnet18_audio_feat_v1_h2.npy
saved for /work/oarong

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2016-2017/2016-12-10 - 22-45 Real Madrid 3 - 2 Dep. La Coruna/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2016-2017/2017-01-08 - 22-45 Villarreal 1 - 1 Barcelona/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2016-2017/2017-01-08 - 22-45 Villarreal 1 - 1 Barcelona/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2016-2017/2017-01-14 - 18-15 Barcelona 5 - 0 Las Palmas/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2016-2017/2017-01-14 - 18-15 Barcelona 5 - 0 Las Palmas/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2016-2017/2017-01-15 - 22-45 Sevilla 2 - 1 Real Madrid/resnet18_audio_feat_v1_h1.npy
saved for /work/o

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2016-2017/2017-05-21 - 21-00 Malaga 0 - 2 Real Madrid/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2016-2017/2017-05-21 - 21-00 Malaga 0 - 2 Real Madrid/resnet18_audio_feat_v1_h2.npy
CPU times: user 4h 47min 55s, sys: 3min 57s, total: 4h 51min 53s
Wall time: 2h 28min 3s


# validation set

In [177]:
%%time
for i in range(len(valid_set_all)):
    data = valid_set_all[i]['ms']
    data = pad_by_wsize(w,data)
    l = data.size()[1]
    model = model.to(device)
    model = model.eval()
    res = list()
    save_path = root_dir+valid_set_all[i]['vidpath']+"/resnet18_audio_feat_v1_h"+str(valid_set_all[i]['half'])+".npy"
    with torch.no_grad():
        for left_idx in range(0,l-(w*40),20):
            inputs = normalize_data(data,(left_idx,left_idx+(w*40))).to(device).unsqueeze(0).unsqueeze(0)
            res.append(model(inputs).cpu())
    k = np.vstack(res)
    np.save(save_path,k)
    print(f"saved for {save_path}")

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-04-11 - 19-30 Burnley 0 - 1 Arsenal/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-04-11 - 19-30 Burnley 0 - 1 Arsenal/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2015-08-30 - 18-00 Swansea 2 - 1 Manchester United/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2015-08-30 - 18-00 Swansea 2 - 1 Manchester United/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2015-09-26 - 17-00 Leicester 2 - 5 Arsenal/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2015-2016/2015-09-26 - 17-00 Leicester 2 - 5 Arsenal/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/soun

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-15 - 21-45 Paris SG 1 - 3 Barcelona/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-15 - 21-45 Paris SG 1 - 3 Barcelona/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-21 - 21-45 Barcelona 2 - 0 Paris SG/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2014-2015/2015-04-21 - 21-45 Barcelona 2 - 0 Paris SG/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2015-2016/2015-09-15 - 21-45 PSV 2 - 1 Manchester United/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/europe_uefa-champions-league/2015-2016/2015-09-1

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/germany_bundesliga/2014-2015/2015-05-02 - 16-30 Hoffenheim 1 - 1 Dortmund/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/germany_bundesliga/2014-2015/2015-05-02 - 16-30 Hoffenheim 1 - 1 Dortmund/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/germany_bundesliga/2015-2016/2015-09-20 - 18-30 Dortmund 3 - 0 Bayer Leverkusen/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/germany_bundesliga/2015-2016/2015-09-20 - 18-30 Dortmund 3 - 0 Bayer Leverkusen/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/germany_bundesliga/2015-2016/2015-10-04 - 18-30 Bayern Munich 5 - 1 Dortmund/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/germany_bundesliga/2015-2016/2015-10-04 - 18-30 Bayern Munich 5 - 1 Dortmund/resn

saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2014-2015/2015-04-11 - 17-00 Real Madrid 3 - 0 Eibar/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2014-2015/2015-04-11 - 21-00 Sevilla 2 - 2 Barcelona/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2014-2015/2015-04-11 - 21-00 Sevilla 2 - 2 Barcelona/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2014-2015/2015-04-28 - 21-00 Barcelona 6 - 0 Getafe/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2014-2015/2015-04-28 - 21-00 Barcelona 6 - 0 Getafe/resnet18_audio_feat_v1_h2.npy
saved for /work/oarongve/data/sound_dataset/SoccerNet-code/data/spain_laliga/2014-2015/2015-05-02 - 19-00 Atl. Madrid 0 - 0 Ath Bilbao/resnet18_audio_feat_v1_h1.npy
saved for /work/oarongve/data/sound_d

In [179]:
test_set_all.generate_mel_spectrograms(load_features=False)

Already loaded


# test set

In [16]:
test_set_all = soccernet_dataset_generic(npy_file=test_list,root_dir=root_dir,lang='all',lang_dict=langpath_test)

In [19]:
test_set_all = soccernet_dataset_generic(npy_file=test_list,root_dir=root_dir,lang='all',lang_dict=langpath_test)
test_set_all.load_waves()
test_set_all.generate_mel_spectrograms(load_features=True)
test_set_all.load_resnet_features()
test_set_all.load_audio_resnet()

KeyboardInterrupt: 

In [ ]:
%%time
for i in range(len(test_set_all)):
    data = test_set_all[i]['ms']
    data = pad_by_wsize(w,data)
    l = data.size()[1]
    model = model.to(device)
    model = model.eval()
    res = list()
    save_path = root_dir+test_set_all[i]['vidpath']+"/resnet18_audio_feat_v1_h"+str(test_set_all[i]['half'])+".npy"
    with torch.no_grad():
        for left_idx in range(0,l-(w*40),20):
            inputs = normalize_data(data,(left_idx,left_idx+(w*40))).to(device).unsqueeze(0).unsqueeze(0)
            res.append(model(inputs).cpu())
    k = np.vstack(res)
    np.save(save_path,k)
    print(f"saved for {save_path}")

In [146]:
root_dir+train_set_all[0]['vidpath']+"/resnet18_audio_feat_v1.npy"

'/work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/resnet18_audio_feat_v1.npy'

In [ ]:
model = model.cuda()

In [148]:
p = root_dir+train_set_all[0]['vidpath']+"/resnet18_audio_feat_v1.npy"

In [150]:
np.save(p,res)

ValueError: only one element tensors can be converted to Python scalars

In [153]:
k = np.vstack(res)

In [159]:
k.shape

(5392, 512)

In [160]:
np.save(p,k)

In [161]:
l = np.load(p)

In [162]:
l.shape

(5392, 512)